In [67]:
import numpy as np

cards = [(i,j) for j in '♣♦♥♠' for i in range(2, 15)]

# если у отбивающегося осталась 1 карта, то на него нельзя ходить,
# т.е. в общем случае подкидывать можно > 6 карт (например, когда до этого отбивающийся взял)

In [173]:
class Agent:
    def __init__(self, ind):
        self.reset(None)
        self.ind = ind
    
    def reset(self, hand):
        self.hand = hand
        # карты, которые видели другие игроки
        self.public = set()
    
    # возвращает кол-во карт до полной руки
    def need_cards(self):
        return max(0, 6-len(self.hand))

    def min_trump(self, trump):
        return min([i[0] for i in self.hand if i[1]==trump] + [15]) # 15 - нет козыря
    
    def drop(self, table, trump):
        """
        возвращает набор карт и козырей, которые можно подкинуть
        """
        s = set()
        for k,v in table.items():
            s.add(k[0])
            if v:
                s.add(v[0])
        t, t1 = [], []
        for i in self.hand:
            if i[0] in s:
                if i[1] == trump:
                    t1.append(i)
                else:
                    t.append(i)
        return (sorted(t), sorted(t1))
    

    def pick_up_cards(self, table):
        for k,v in table.items():
            self.hand.add(k)
            if v:
                self.hand.add(v)

    def __repr__(self):
        return f'Agent{self.ind} {self.hand}'
    
class Pair:
    ind = 0
    def __init__(self, agents, trump):
        self.agents = agents
        self.trump = trump

    def swap(self):
        self.agents = self.agents[::-1]
    
    def cards_count(self):
        return len(self.agents[0].hand)

    def pick_up_cards(self, table):
        self.agents[0].pick_up_cards(table)

    def __repr__(self):
        return f'Pair{self.ind}\n' + '\n'.join(list(map(str, self.agents)))
    
    # выбор наименьшей карты для первого хода
    def first_hit(self):
        a = self.agents[0]
        t = [i for i in a.hand if i[1]!=self.trump]
        if not t:
            t = a.hand
        
        card = min(t)
        a.hand.remove(card)
        return card
    
    # попытка отбить карты
    def beat(self, table):
        ag = self.agents[0]
        hand = []
        trumps = []
        
        for i in ag.hand:
            if i[1] == self.trump:
                trumps.append(i)
            else:
                hand.append(i)
        hand.sort() # список обычных карт
        trumps.sort() # список козырей
        
        # вспомогательная функция
        def min_to_beat(hand, card):
            for i in hand:
                if i[1] == card[1] and i[0] > card[0]:
                    return i
            return None
        
        cards = []
        for k,v in table.items():
            if not v:
                t = None
                if k[1] == self.trump:
                    t = min_to_beat(trumps, k)
                    if not t:
                        return False
                    trumps.remove(t)
                else:
                    t = min_to_beat(hand, k)
                    if not t:
                        if trumps:
                            t = trumps[0]
                            trumps.pop(0)
                        else:
                            return False
                    else:
                        hand.remove(t)
                table[k] = t
                cards.append(t)
        
        ag.hand -= set(cards)
        return True

    # попытка перевести карты
    def transfer(self, table, opp):
        if len(opp.hand) < len(table) + 1:
            # print('Перевод невозможен')
            return False
        a = self.agents[0]
        t0, t1 = a.drop(table, self.trump)
        if t0:
            table[t0[0]] =  None
            a.hand.remove(t0[0])
            # print('Перевод')
            return True
        return False

    # подкинуть карты
    def toss_cards(self, table, opp, verbose=0):
        n = len(opp.hand)-1
        m = n
        
        # заходящий игрок
        a = self.agents[0]
        t0, t1 = a.drop(table, self.trump)
        k = min(n, len(t0))
        indices = np.random.choice(len(t0), k, replace=False)
        for i in indices:
            table[t0[i]] = None
            n-=1
            a.hand.remove(t0[i])
        
        if len(self.agents)==1:
            return bool(m-n)
        # напарник
        a = self.agents[1]
        t0, t1 = a.drop(table, self.trump)
        k = min(n, len(t0))
        indices = np.random.choice(len(t0), k, replace=False)
        for i in indices:
            table[t0[i]] = None
            n-=1
            a.hand.remove(t0[i])
        
        return bool(m-n) # подкинули карты или нет



class Pair1(Pair):
    ind = 1

    def first_hit(self):
        a = self.agents[0]
        # t = [i for i in a.hand if i[1]!=self.trump]
        # if not t:
        t = a.hand
        i = np.random.choice(len(t))
        card = (list(t)[i])
        a.hand.remove(card)
        return card


class Pair2(Pair):
    ind = 2

    

    def toss_cards(self, table, opp, verbose=0):
        n = len(opp.hand)-1
        m = n
        
        # заходящий игрок
        a = self.agents[0]
        t0, t1 = a.drop(table, self.trump)
        t0 += t1
        k = min(n, len(t0))
        indices = np.random.choice(len(t0), k, replace=False)
        for i in indices:
            table[t0[i]] = None
            n-=1
            a.hand.remove(t0[i])
        
        if len(self.agents)==1:
            return bool(m-n)
        # напарник
        a = self.agents[1]
        t0, t1 = a.drop(table, self.trump)
        t0 += t1
        k = min(n, len(t0))
        indices = np.random.choice(len(t0), k, replace=False)
        for i in indices:
            table[t0[i]] = None
            n-=1
            a.hand.remove(t0[i])
        
        return bool(m-n)


In [174]:
class Env:
    def __init__(self):
        self.bito = set()
        self.deck = []
        self.trump = ''
        self.table = {}

        self.agents = [Agent(i) for i in range(4)]
        self.attack = None
        self.defend = None
    
    # начало новой игры
    def reset(self):
        self.bito = set()
        self.table = {}
        
        np.random.shuffle(cards)

        for i in range(4): # раздача
            self.agents[i].reset(set(cards[i*6:(i+1)*6]))

        self.deck = cards[24:]
        self.trump = cards[-1][1]
        
        a = Pair1(self.agents[:2], self.trump)
        b = Pair2(self.agents[2:], self.trump)

        t = np.argmin([i.min_trump(self.trump) for i in self.agents])
        if t < 2 and t == 1:
            a.swap()
        elif t > 1:
            a, b = b, a
            if t == 3:
                a.swap()
        
        self.attack = a # атака
        self.defend = b # защита
        # return (a1,a2), (a3,a4)
    
    # смена ролей при переводе или новом раунде
    def rotate(self, flag=0):
        """
        flag = 1 - отбивающий взял
        """
        self.attack.swap()
        if flag:
            self.defend.swap()
        else:
            self.attack, self.defend = self.defend, self.attack
    
    # восполнение карт
    def fill_hands(self):
        """
        - первым берет атакующий и его помощник
        - далее берет помощник защитника, т.к. мог быть перевод
        - в конце берет защитник

        если кто-то победит, то возвращаем номер команды (иначе 0)
        """
        
        i = 0
        while i < len(self.attack.agents):
            ag = self.attack.agents[i]
            t = ag.need_cards()
            if t == 6 and not self.deck:
                self.attack.agents.pop(i)
                i-=1
            elif t > 0:
                t = min(t, len(self.deck))
                ag.hand |= set(self.deck[:t])
                self.deck = self.deck[t:]
            i+=1
        
        if not self.attack.agents:
            return self.attack.ind
        

        i = 0
        self.defend.swap() # для простоты два раза их свапнем
        while i < len(self.defend.agents):
            ag = self.defend.agents[i]
            t = ag.need_cards()
            if t == 6 and not self.deck:
                self.defend.agents.pop(i)
                i-=1
            elif t > 0:
                t = min(t, len(self.deck))
                ag.hand |= set(self.deck[:t])
                self.deck = self.deck[t:]
            i+=1
        self.defend.swap() # обратный свап

        if not self.defend.agents:
            return self.defend.ind
        
        return 0 # пока нет победителя


    def play(self, verbose=0):
        # в первый ход нужно положить карту на стол вручную
        # должна быть проверка, можно ли подкидывать (кол-во карт > 1) и отбил ли карты отбивающийся
        # после раунда сначала берут карты, а потом смена ролей
        if not self.table:
            if verbose:
                print(' Атака:', self.attack)
            card = self.attack.first_hit()
            if verbose:
                print('\n Защита:', self.defend)
                print('\n Ход:', card)
            self.table[card] = None
        
        # print()
        first_hit = 1
        t = self.defend.beat(self.table)
        if t:
            first_hit = 0
        while t and self.defend.cards_count() > 1:
            f = self.attack.toss_cards(self.table, self.defend.agents[0])
            if not f: # если не подкинули то отбой
                break
            t = self.defend.beat(self.table)
        
        # не смог отбить на первой итерации
        if not t and first_hit:
            t = self.defend.transfer(self.table, self.attack.agents[-1])
            if t:
                self.rotate()
                return self.play(verbose)

        if verbose:
            print('Стол:', self.table)
        if not t:
            if verbose:
                print(f'Agent{self.defend.agents[0].ind} берет карты')
            self.defend.pick_up_cards(self.table)
        else:
            if verbose:
                print(f'Agent{self.defend.agents[0].ind} отбился')
            for k,v in self.table.items():
                self.bito.add(k)
                self.bito.add(v)
        # тут еще нужно обновлять знания
        if verbose:
            print('bito:', self.bito)
        self.table = {}
        
        if verbose:
            print('\nАгенты восполняют карты из колоды\n')
        res = self.fill_hands()
        if verbose:
            print(f'итог игры: {'победитель - команда ' + str(res) if res else 'победитель пока не определен'}')
        
        if res:
            return res

        self.rotate(not t)
        if verbose:
            print('\nПосле смены ролей:\n')
            print(' Атака:', self.attack)
            print()
            print(' Защита:', self.defend)
        
        return self.play(verbose)
        # print(self.attack)


env = Env()

In [175]:
d = {1:0, 2:0}
for i in range(1000):
    env.reset()
    d[env.play()] += 1
print(d)

{1: 235, 2: 765}


In [133]:
env.reset()
print('Козырь:',env.trump)
env.play(verbose=1)

Козырь: ♠
 Атака: Pair1
Agent0 {(2, '♦'), (13, '♦'), (5, '♠'), (10, '♦'), (6, '♦'), (11, '♠')}
Agent1 {(6, '♣'), (10, '♠'), (6, '♠'), (11, '♥'), (14, '♥'), (11, '♦')}

 Защита: Pair2
Agent2 {(13, '♠'), (14, '♦'), (9, '♣'), (7, '♥'), (14, '♣'), (5, '♥')}
Agent3 {(12, '♦'), (11, '♣'), (12, '♥'), (2, '♣'), (8, '♣'), (5, '♣')}

 Ход: (2, '♦')
Стол: {(2, '♦'): (14, '♦'), (14, '♥'): (13, '♠'), (13, '♦'): None}
Agent2 берет карты
bito: set()

Агенты восполняют карты из колоды

итог игры: победитель пока не определен

После смены ролей:

 Атака: Pair1
Agent1 {(6, '♣'), (10, '♠'), (6, '♠'), (11, '♥'), (12, '♣'), (11, '♦')}
Agent0 {(5, '♠'), (10, '♦'), (6, '♦'), (7, '♣'), (9, '♠'), (11, '♠')}

 Защита: Pair2
Agent3 {(12, '♦'), (11, '♣'), (12, '♥'), (2, '♣'), (8, '♣'), (5, '♣')}
Agent2 {(13, '♠'), (2, '♦'), (13, '♦'), (14, '♦'), (9, '♣'), (7, '♥'), (14, '♥'), (14, '♣'), (5, '♥')}
 Атака: Pair1
Agent1 {(6, '♣'), (10, '♠'), (6, '♠'), (11, '♥'), (12, '♣'), (11, '♦')}
Agent0 {(5, '♠'), (10, '♦'), (6,

2